In [2]:
%matplotlib inline

In [11]:
!pip install astroquery

     |████████████████████████████████| 6.5MB 4.3MB/s 
     |████████████████████████████████| 2.6MB 56.6MB/s 
  Created wheel for astroquery: filename=astroquery-0.4.1-cp36-none-any.whl size=3831873 sha256=c9e493b4a265ddff9a44d6d19bdc372d5fcc5c2f66d933937a9634700c726492
  Stored in directory: /root/.cache/pip/wheels/88/f8/b7/a254cd96e808f708bc0b7d755a8e095c56fbbe94099d7b464f
Successfully built astroquery


In [12]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
import torch.optim as optim

import pandas as pd
import numpy as np

import tqdm
import matplotlib.pyplot as plt

from astropy.time import Time
from astroquery.jplhorizons import Horizons

import re

In [226]:
# Загружаем файлы и готовим некоторые списки

df = pd.read_csv('гороскопы мейла с фичами.csv', sep=';') # в этом файле есть координаты, но только по 2 первых слова прогноза

signs = ['Овен',	'Телец',	'Близнецы',	'Рак',	'Лев',	'Дева',	'Весы',	'Скорпион',	'Стрелец',	'Козерог',	'Водолей',	'Рыбы']
planets = {'Sun': 10,  'Mercury': 199, 'Venus': 299, 'Mars': 499, 'Jupiter': 599, 'Saturn': 699, 'Uranus': 799, 'Neptune': 899, 'Pluto': 999}
features = []
for k, v in planets.items():
  features += [k+'_x', k+'_y', k+'_z', k+'_vx', k+'_vy', k+'_vz', k+'_l', k+'_ry', k+'_rr']

In [227]:
df_orig = pd.read_csv('df_three.csv', sep=';') # это оригинальный файл с соскрапленными гороскопами

def conv_date(d):
  d = d.split()
  year = d[-1]
  day = d[0]
  if d[1] == 'января':
    month = '01'
  elif d[1] == 'февраля':
    month = '02'
  elif d[1] == 'марта':
    month = '03'
  elif d[1] == 'апреля':
    month = '04'
  elif d[1] == 'мая':
    month = '05'
  elif d[1] == 'июня':
    month = '06'
  elif d[1] == 'июля':
    month = '07'
  elif d[1] == 'августа':
    month = '08'
  elif d[1] == 'сентября':
    month = '09'
  elif d[1] == 'октября':
    month = '10'
  elif d[1] == 'ноября':
    month = '11'
  elif d[1] == 'декабря':
    month = '12'

  return '-'.join([year, month, day])

df_orig = df_orig[['date'] + signs].copy()
df_orig.fillna('0')
df_orig['date'] =  df_orig['date'].apply(lambda x: conv_date(x))
df_orig['date_for_epoch'] =  df_orig['date'].apply(lambda x: Time(x).jd)

start_sign_cols = []
for sign in signs:
  start_sign_cols.append('start_' + sign)
  df_orig['start_' + sign] = df_orig[sign].apply(lambda x: ' '.join(re.sub(r'[^\w\s]','',x).split()[0:5]).lower() if type(x) == str else x)
  df_orig[sign] = df_orig[sign].apply(lambda x: ' '.join(re.sub(r'[^\w\s]','',x).split()).lower() if type(x) == str else x)

In [228]:
df[start_sign_cols] = df_orig[start_sign_cols]

In [229]:
df_orig.head(2)

,date,Овен,Телец,Близнецы,Рак,Лев,Дева,Весы,Скорпион,Стрелец,Козерог,Водолей,Рыбы,date_for_epoch,start_Овен,start_Телец,start_Близнецы,start_Рак,start_Лев,start_Дева,start_Весы,start_Скорпион,start_Стрелец,start_Козерог,start_Водолей,start_Рыбы
0,2020-10-18,насыщенный интересный день за любые дела вы бе...,вам может быть нелегко отстоять свои интересы ...,тот самый день когда ваши поступки вызывают вс...,для самостоятельной работы это не самый подход...,вы без труда завоевываете симпатии новых знако...,первая половина дня проходит спокойно можно де...,благоприятный день для общения удается произве...,не спешите это не тот день когда можно за счит...,напряженный день будьте осторожны и внимательн...,с эмоциональной точки зрения день будет доволь...,утром стоит быть осторожнее поскольку в это вр...,здравый смысл и практичность помогают избежать...,2459140.5,насыщенный интересный день за любые,вам может быть нелегко отстоять,тот самый день когда ваши,для самостоятельной работы это не,вы без труда завоевываете симпатии,первая половина дня проходит спокойно,благоприятный день для общения удается,не спешите это не тот,напряженный день будьте осторожны и,с эмоциональной точки зрения день,утром стоит быть осторожнее поскольку,здравый смысл и практичность помогают
1,2020-10-17,будьте практичны этот день может положить нача...,день складывается удачно хотя порой вы сами се...,благоприятный день для любых встреч и перегово...,день подходит для решения любых вопросов в том...,вы способны на многое и сами это понимаете но ...,утро потребует осторожности в делах и сдержанн...,удачный день открываются новые возможности и в...,утро проходит очень плодотворно в это время вы...,вы решите многие проблемы если расстанетесь с ...,день складывается напряженно но влияние благоп...,неплохой день подходящий для любого общения вы...,день хоть и не лишен сложных моментов но все ж...,2459139.5,будьте практичны этот день может,день складывается удачно хотя порой,благоприятный день для любых встреч,день подходит для решения любых,вы способны на многое и,утро потребует осторожности в делах,удачный день открываются новые возможности,утро проходит очень плодотворно в,вы решите многие проблемы если,день складывается напряженно но влияние,неплохой день подходящий для любого,день хоть и не лишен


In [230]:
df.head(2)

,Unnamed: 0,date,date_for_epoch,start_Овен,start_Телец,start_Близнецы,start_Рак,start_Лев,start_Дева,start_Весы,start_Скорпион,start_Стрелец,start_Козерог,start_Водолей,start_Рыбы,Sun_x,Sun_y,Sun_z,Sun_vx,Sun_vy,Sun_vz,Sun_l,Sun_ry,Sun_rr,Mercury_x,Mercury_y,Mercury_z,Mercury_vx,Mercury_vy,Mercury_vz,Mercury_l,Mercury_ry,Mercury_rr,Venus_x,Venus_y,Venus_z,Venus_vx,Venus_vy,Venus_vz,Venus_l,...,Jupiter_vz,Jupiter_l,Jupiter_ry,Jupiter_rr,Saturn_x,Saturn_y,Saturn_z,Saturn_vx,Saturn_vy,Saturn_vz,Saturn_l,Saturn_ry,Saturn_rr,Uranus_x,Uranus_y,Uranus_z,Uranus_vx,Uranus_vy,Uranus_vz,Uranus_l,Uranus_ry,Uranus_rr,Neptune_x,Neptune_y,Neptune_z,Neptune_vx,Neptune_vy,Neptune_vz,Neptune_l,Neptune_ry,Neptune_rr,Pluto_x,Pluto_y,Pluto_z,Pluto_vx,Pluto_vy,Pluto_vz,Pluto_l,Pluto_ry,Pluto_rr
0,0,2020-10-18,2459140.5,насыщенный интересный день за любые,вам может быть нелегко отстоять,тот самый день когда ваши,для самостоятельной работы это не,вы без труда завоевываете симпатии,первая половина дня проходит спокойно,благоприятный день для общения удается,не спешите это не тот,напряженный день будьте осторожны и,с эмоциональной точки зрения день,утром стоит быть осторожнее поскольку,здравый смысл и практичность помогают,-0.904546,-0.417749,0.000024,0.007499,-0.015560,1.876724e-07,0.005754,0.996353,-0.000283,-0.54520,-0.462993,-0.036637,0.005571,0.013612,0.002561,0.004136,0.716204,-0.013171,-1.157731,0.254497,0.023859,-0.011498,-0.022803,0.000997,0.006848,...,-0.000161,0.029598,5.124750,0.015327,4.254547,-8.982140,-0.056410,0.011974,-0.012692,-0.000228,0.057403,9.938973,0.016597,14.631491,11.826922,-0.155757,0.005041,-0.012651,0.000043,0.108663,18.814379,-0.004033,28.514965,-5.883028,-0.565512,0.008058,-0.012450,-0.000076,0.168189,29.121007,0.010407,12.935142,-31.622146,-0.663183,0.010449,-0.014963,-0.000927,0.197360,34.171886,0.017819
1,1,2020-10-17,2459139.5,будьте практичны этот день может,день складывается удачно хотя порой,благоприятный день для любых встреч,день подходит для решения любых,вы способны на многое и,утро потребует осторожности в делах,удачный день открываются новые возможности,утро проходит очень плодотворно в,вы решите многие проблемы если,день складывается напряженно но влияние,неплохой день подходящий для любого,день хоть и не лишен,-0.911909,-0.402126,0.000024,0.007225,-0.015684,3.191686e-07,0.005756,0.996636,-0.000283,-0.55172,-0.476378,-0.039085,0.007454,0.013131,0.002334,0.004216,0.729972,-0.014328,-1.145999,0.277093,0.022853,-0.011965,-0.022388,0.001016,0.006811,...,-0.000161,0.029509,5.109411,0.015350,4.242709,-8.969385,-0.056182,0.011702,-0.012818,-0.000228,0.057307,9.922379,0.016592,14.626586,11.839634,-0.155800,0.004768,-0.012774,0.000043,0.108687,18.818561,-0.004331,28.507044,-5.870516,-0.565436,0.007785,-0.012574,-0.000076,0.168130,29.110723,0.010160,12.924825,-31.607126,-0.662256,0.010184,-0.015076,-0.000926,0.197257,34.154064,0.017824


In [235]:
df = df.drop(['date',	'date_for_epoch'], axis=1)

In [236]:
df_orig = df_orig.dropna()

In [42]:
from gensim.models.word2vec import Word2Vec
from multiprocessing import cpu_count
import gensim.downloader as api

data_part = [d.split() for d in list(df_orig[signs].values.reshape(3708))]
model = Word2Vec(data_part, min_count=1, size= 50,workers=3, window =3, sg = 1)

In [45]:
model['день'], model.most_similar('день'), model.similar_by_vector(model['день'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similar_by_vector` (Method will be removed in 4.0.0, use self.wv.similar_by_vector() instead).
  """Entry point for launching an IPython ker

(array([-0.7106669 , -0.23918846, -0.68920034, -0.81857973,  0.09385717,
         0.37405047,  0.9113585 ,  1.1342427 , -0.00615251,  0.20109695,
         0.15533085,  0.15152927,  0.19767262, -0.35505208,  0.280773  ,
         1.0992775 , -1.0560215 ,  0.4510842 , -0.70653206, -0.32093382,
        -0.5315345 , -0.23144445, -0.12122159,  0.20167024,  0.3195589 ,
        -0.7950856 , -0.85638386,  0.46323964,  1.0144122 , -0.57556224,
         0.09162327,  0.00692166, -0.71455896,  0.46958956, -0.30770338,
         0.25409475,  0.777749  , -0.35977176, -0.990333  , -0.02895246,
         0.21992087, -0.8981342 ,  0.5372583 ,  0.4615086 ,  0.23233661,
         0.31055608, -0.36808327,  0.00215674,  0.40252703, -1.1655388 ],
       dtype=float32),
 [('он', 0.8763746023178101),
  ('плодотворный', 0.8088374137878418),
  ('благоприятный', 0.7873494625091553),
  ('удачный', 0.7851084470748901),
  ('период', 0.7841936349868774),
  ('спокойный', 0.7802112102508545),
  ('вполне', 0.77904224395751

In [44]:
model.save("word2vec.model")
# model = Word2Vec.load("word2vec.model")

In [50]:
# Загруженный файл переводим из горизонтального вида в вертикальный, чтобы все знаки зодиака шли одним столбцом start (первые 5 слов прогноза)
# Класс знака зодиака задается в столбцах cls
# Все прочие столбцы - это координаты планет.

data = df[['start_Овен'] + features].copy()
cls_cols = [el + '_cls' for el in signs]
data[cls_cols] = [1,0,0,0,0,0,0,0,0,0,0,0]
data = data.rename(columns={'start_Овен': 'start'})
for i, sign in enumerate(signs):
  if sign != 'Овен':
    data_part = df[['start_' + sign] + features].copy()
    cls_ohe = [0 for _ in range(12)]
    cls_ohe[i] = 1
    data_part[cls_cols] = cls_ohe
    data_part = data_part.rename(columns={'start_' + sign: 'start'})
    data = pd.concat([data, data_part], ignore_index=True)
data[::500]

,start,Sun_x,Sun_y,Sun_z,Sun_vx,Sun_vy,Sun_vz,Sun_l,Sun_ry,Sun_rr,Mercury_x,Mercury_y,Mercury_z,Mercury_vx,Mercury_vy,Mercury_vz,Mercury_l,Mercury_ry,Mercury_rr,Venus_x,Venus_y,Venus_z,Venus_vx,Venus_vy,Venus_vz,Venus_l,Venus_ry,Venus_rr,Mars_x,Mars_y,Mars_z,Mars_vx,Mars_vy,Mars_vz,Mars_l,Mars_ry,Mars_rr,Jupiter_x,Jupiter_y,Jupiter_z,...,Saturn_rr,Uranus_x,Uranus_y,Uranus_z,Uranus_vx,Uranus_vy,Uranus_vz,Uranus_l,Uranus_ry,Uranus_rr,Neptune_x,Neptune_y,Neptune_z,Neptune_vx,Neptune_vy,Neptune_vz,Neptune_l,Neptune_ry,Neptune_rr,Pluto_x,Pluto_y,Pluto_z,Pluto_vx,Pluto_vy,Pluto_vz,Pluto_l,Pluto_ry,Pluto_rr,Овен_cls,Телец_cls,Близнецы_cls,Рак_cls,Лев_cls,Дева_cls,Весы_cls,Скорпион_cls,Стрелец_cls,Козерог_cls,Водолей_cls,Рыбы_cls
0,насыщенный интересный день за любые,-0.904546,-0.417749,0.000024,0.007499,-0.015560,1.876724e-07,0.005754,0.996353,-0.000283,-0.545200,-0.462993,-0.036637,0.005571,0.013612,0.002561,0.004136,0.716204,-0.013171,-1.157731,0.254497,0.023859,-0.011498,-0.022803,0.000997,0.006848,1.185614,0.006353,0.400402,0.142048,-0.020257,0.002517,-0.001505,0.000417,0.002457,0.425335,0.001847,1.671105,-4.844475,-0.039216,...,0.016597,14.631491,11.826922,-0.155757,0.005041,-0.012651,0.000043,0.108663,18.814379,-0.004033,28.514965,-5.883028,-0.565512,0.008058,-0.012450,-0.000076,0.168189,29.121007,0.010407,12.935142,-31.622146,-0.663183,0.010449,-0.014963,-0.000927,0.197360,34.171886,0.017819,1,0,0,0,0,0,0,0,0,0,0,0
500,насыщенный и беспокойный день трудно,0.996110,0.009777,-0.000005,0.000116,0.017274,-1.196140e-06,0.005753,0.996158,0.000285,0.766874,-0.390431,-0.011679,0.018855,0.004639,-0.002753,0.004971,0.860621,0.014734,0.507749,0.535535,0.035391,-0.014776,0.003401,0.000668,0.004267,0.738823,-0.007657,0.602822,-1.426898,-0.020460,0.014141,0.014782,-0.000398,0.008947,1.549145,-0.008107,2.111377,-5.067446,-0.003870,...,-0.014609,17.037216,11.630479,-0.164617,-0.002214,0.020277,0.000040,0.119144,20.629151,0.009604,30.290257,-6.110114,-0.549179,0.000745,0.020369,-0.000079,0.178494,30.905254,-0.003295,14.210949,-31.316886,-0.469904,0.003090,0.017822,-0.000930,0.198641,34.393593,-0.014938,0,1,0,0,0,0,0,0,0,0,0,0
1000,начинайте день с важных дел,-0.747984,0.684137,-0.000032,-0.011334,-0.012625,9.393918e-07,0.005854,1.013668,-0.000157,-0.809734,0.988610,0.030512,-0.044552,-0.017190,0.002675,0.007383,1.278260,0.014991,-0.022607,0.682313,-0.041916,-0.011370,0.007508,0.000279,0.003950,0.683973,0.007849,0.552370,0.218916,-0.041681,-0.006085,0.001747,0.000173,0.003440,0.595629,-0.005014,1.370762,-4.005721,-0.027957,...,0.005800,14.956193,12.726945,-0.158731,-0.013750,-0.009684,0.000043,0.113425,19.638942,-0.016748,28.632149,-4.995577,-0.560248,-0.010752,-0.009520,-0.000077,0.167895,29.070081,-0.008952,12.887886,-30.561460,-0.600056,-0.008387,-0.012047,-0.000914,0.191592,33.173189,0.007857,0,0,0,1,0,0,0,0,0,0,0,0
1500,не самый плодотворный но вполне,-0.007773,-0.983687,0.000048,0.017489,-0.000077,-3.480059e-07,0.005681,0.983718,-0.000062,-0.273555,-1.355781,-0.005976,0.034692,-0.015107,-0.002807,0.007988,1.383117,0.007959,0.702987,-1.131395,-0.042995,0.021505,0.019636,0.000038,0.007697,1.332702,-0.005328,-1.406163,-1.762106,0.018047,0.024818,-0.011106,-0.000411,0.013021,2.254471,-0.006802,0.444079,-6.195340,0.011585,...,0.005766,16.240495,10.364949,-0.168246,0.015215,0.002965,0.000040,0.111276,19.266918,0.014420,29.228476,-7.381753,-0.542065,0.018147,0.003011,-0.000079,0.174138,30.151084,0.016856,12.939145,-32.359162,-0.387340,0.020478,0.000455,-0.000929,0.201290,34.852358,0.007190,0,0,0,0,1,0,0,0,0,0,0,0
2000,на первую половину дня лучше,0.474472,0.894509,-0.000044,-0.014925,0.008129,2.441673e-07,0.005848,1.012556,0.000188,0.167886,1.060036,0.041607,-0.034068,-0.015443,-0.000170,0.006203,1.074054,-0.020573,0.055337,0.303888,0.016039,0.001429,-0.003670,-0.001105,0.001786,0.309301,-0.003407,0.980634,-0.428477,-0.040183,-0.001326,0.014332,-0.000203,0.006185,1.070911,-0.006940,2.043494,-4.037009,-0.014666,...,-0.014400,16.367610,12.703575,-

In [237]:
data = data.dropna()

In [238]:
# Переводим входные слова в числа согласно словарю

def convert(string_seq):
  return np.array([model[x] for x in string_seq.split(' ')]).reshape(250)

data['start_num'] = data['start'].apply(lambda x: convert(x))
data['start_num_1'] = data['start'].apply(lambda x: np.array(model[x.split(' ')[0]]))
data['start_num_2'] = data['start'].apply(lambda x: np.array(model[x.split(' ')[1]]))
data['start_num_3'] = data['start'].apply(lambda x: np.array(model[x.split(' ')[2]]))
data['start_num_4'] = data['start'].apply(lambda x: np.array(model[x.split(' ')[3]]))
data['start_num_5'] = data['start'].apply(lambda x: np.array(model[x.split(' ')[4]]))
data.head(3)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__g

,start,Sun_x,Sun_y,Sun_z,Sun_vx,Sun_vy,Sun_vz,Sun_l,Sun_ry,Sun_rr,Mercury_x,Mercury_y,Mercury_z,Mercury_vx,Mercury_vy,Mercury_vz,Mercury_l,Mercury_ry,Mercury_rr,Venus_x,Venus_y,Venus_z,Venus_vx,Venus_vy,Venus_vz,Venus_l,Venus_ry,Venus_rr,Mars_x,Mars_y,Mars_z,Mars_vx,Mars_vy,Mars_vz,Mars_l,Mars_ry,Mars_rr,Jupiter_x,Jupiter_y,Jupiter_z,...,Uranus_vz,Uranus_l,Uranus_ry,Uranus_rr,Neptune_x,Neptune_y,Neptune_z,Neptune_vx,Neptune_vy,Neptune_vz,Neptune_l,Neptune_ry,Neptune_rr,Pluto_x,Pluto_y,Pluto_z,Pluto_vx,Pluto_vy,Pluto_vz,Pluto_l,Pluto_ry,Pluto_rr,Овен_cls,Телец_cls,Близнецы_cls,Рак_cls,Лев_cls,Дева_cls,Весы_cls,Скорпион_cls,Стрелец_cls,Козерог_cls,Водолей_cls,Рыбы_cls,start_num,start_num_1,start_num_2,start_num_3,start_num_4,start_num_5
0,насыщенный интересный день за любые,-0.904546,-0.417749,0.000024,0.007499,-0.015560,1.876724e-07,0.005754,0.996353,-0.000283,-0.545200,-0.462993,-0.036637,0.005571,0.013612,0.002561,0.004136,0.716204,-0.013171,-1.157731,0.254497,0.023859,-0.011498,-0.022803,0.000997,0.006848,1.185614,0.006353,0.400402,0.142048,-0.020257,0.002517,-0.001505,0.000417,0.002457,0.425335,0.001847,1.671105,-4.844475,-0.039216,...,0.000043,0.108663,18.814379,-0.004033,28.514965,-5.883028,-0.565512,0.008058,-0.012450,-0.000076,0.168189,29.121007,0.010407,12.935142,-31.622146,-0.663183,0.010449,-0.014963,-0.000927,0.197360,34.171886,0.017819,1,0,0,0,0,0,0,0,0,0,0,0,"[-0.20097116, -0.39993128, -0.38080287, -0.772...","[-0.20097116, -0.39993128, -0.38080287, -0.772...","[-0.24224697, -0.32531098, -0.29583368, -0.704...","[-0.7106669, -0.23918846, -0.68920034, -0.8185...","[0.8948045, 0.23784168, 0.38576618, 0.47380295...","[0.05232921, -0.0726626, -0.6013601, -0.753021..."
1,будьте практичны этот день может,-0.911909,-0.402126,0.000024,0.007225,-0.015684,3.191686e-07,0.005756,0.996636,-0.000283,-0.551720,-0.476378,-0.039085,0.007454,0.013131,0.002334,0.004216,0.729972,-0.014328,-1.145999,0.277093,0.022853,-0.011965,-0.022388,0.001016,0.006811,1.179244,0.006387,0.397954,0.143586,-0.020673,0.002379,-0.001572,0.000415,0.002446,0.423570,0.001682,1.657301,-4.833002,-0.039055,...,0.000043,0.108687,18.818561,-0.004331,28.507044,-5.870516,-0.565436,0.007785,-0.012574,-0.000076,0.168130,29.110723,0.010160,12.924825,-31.607126,-0.662256,0.010184,-0.015076,-0.000926,0.197257,34.154064,0.017824,1,0,0,0,0,0,0,0,0,0,0,0,"[0.39644277, 0.020643601, 0.0720196, -0.075878...","[0.39644277, 0.020643601, 0.0720196, -0.075878...","[0.23707008, -0.21587744, 0.03597016, -0.24997...","[0.11832371, -0.2798793, -0.7086075, -0.636874...","[-0.7106669, -0.23918846, -0.68920034, -0.8185...","[-0.23870714, -0.26901338, -0.12962128, -0.091..."
2,благоприятный день если вы проявите,-0.918996,-0.386383,0.000023,0.006950,-0.015802,4.764212e-07,0.005758,0.996918,-0.000282,-0.560073,-0.489208,-0.041304,0.009234,0.012507,0.002104,0.004302,0.744790,-0.015276,-1.133805,0.299270,0.021828,-0.012420,-0.021963,0.001034,0.006774,1.172839,0.006421,0.395644,0.145190,-0.021087,0.002240,-0.001634,0.000413,0.002437,0.421971,0.001517,1.643766,-4.821399,-0.038894,...,0.000043,0.108713,18.823040,-0.004627,28.499397,-5.857882,-0.565359,0.007509,-0.012693,-0.000076,0.168072,29.100688,0.009911,12.914777,-31.591994,-0.661335,0.009910,-0.015187,-0.000915,0.197154,34.136241,0.017822,1,0,0,0,0,0,0,0,0,0,0,0,"[-0.28936547, -0.46712843, -0.6835967, -0.6978...","[-0.28936547, -0.46712843, -0.6835967, -0.6978...","[-0.7106669, -0.23918846, -0.68920034, -0.8185...","[0.60200465, -0.55100614, 0.12129481, -0.82037...","[0.33136016, -0.3066615, 0.30393177, -0.716541...","[0.3615729, -0.41036066, -0.25291285, -0.05460..."


In [240]:
data.to_csv('первые 5 слов коорд ohe emb.csv', sep=';')

In [223]:
# Типичные торчевые штуки и полносвязная сеть

class HoroDataset(Dataset):

    def __init__(self, df, col_name):

        self.df = df

        self.X = self.df[features + cls_cols]
        self.y = self.df[[col_name]]

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # Convert idx from tensor to list due to pandas bug (that arises when using pytorch's random_split)
        if isinstance(idx, torch.Tensor):
            idx = idx.tolist()

        return [self.X.iloc[idx].values, torch.tensor([el for el in self.y.iloc[idx].values]).squeeze()] # self.y.iloc[idx].values]
        # return [self.X.iloc[idx].values, self.y[idx]]

class Net(nn.Module):

    def __init__(self, D_in, H=512, D_out=50):
        super().__init__()
        self.fc1 = nn.Linear(D_in, H)
        self.fc1_5 = nn.Linear(H, int(H / 2))
        self.fc2 = nn.Linear(int(H / 2), D_out)
        self.relu = nn.ReLU()
        self.batchnorm1 = nn.BatchNorm1d(H)
        self.dropout = nn.Dropout(p=0.2)

    def forward(self, x):
        x = self.fc1(x)
        x = self.batchnorm1(x)
        x = self.relu(x)
        x = self.fc1_5(x)
        x = self.dropout(x)
        x = self.fc2(x)

        return x.squeeze()

answers = []

for numb_word in range(1, 6):

  dataset = HoroDataset(data, 'start_num_' + str(numb_word))
  train_size = int(0.9 * len(dataset))
  test_size = len(dataset) - train_size
  trainset, testset = random_split(dataset, [train_size, test_size])
  trainloader = DataLoader(trainset, batch_size=128, shuffle=True)
  testloader = DataLoader(testset, batch_size=128, shuffle=False)
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  D_in, H = 93, 372
  net = Net(D_in, H).to(device)
  criterion = nn.MSELoss()
  optimizer = optim.Adam(net.parameters(), weight_decay=0.0001)

  net.train()
  torch.set_grad_enabled(True)
  loss_per_iter = []
  loss_per_batch = []

  for epoch in tqdm.notebook.tqdm(range(5)):

      running_loss = 0.0
      for i, (inputs, labels) in enumerate(trainloader):
          inputs = inputs.to(device)
          labels = labels.to(device)

          optimizer.zero_grad()

          outputs = net(inputs.float())
          loss = criterion(outputs, labels.float())
          loss.backward()
          optimizer.step()

          running_loss += loss.item()
          loss_per_iter.append(loss.item())

      loss_per_batch.append(running_loss / (i + 1))
      running_loss = 0.0

  net.eval()

  torch.set_grad_enabled(False)

  df_pred = data[::100]
  input_tensor = torch.tensor(df_pred[features + cls_cols].values).float()
  out_tensor = net(input_tensor.cuda(device=device)).cpu()
  out = out_tensor.numpy().round()
  answer = []
  for el in out:
    answer.append(model.similar_by_vector(el)[0][0])

  answers.append(answer)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:95: DeprecationWarning: Call to deprecated `similar_by_vector` (Method will be removed in 4.0.0, use self.wv.similar_by_vector() instead).
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [224]:
answers_out = []
for i in range(len(df_pred)):
  s = ' '.join([answers[0][i], answers[1][i], answers[2][i], answers[3][i], answers[4][i]])
  answers_out.append(s)

In [225]:
df_pred['answer'] = answers_out
df_pred[['start', 'answer']]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,start,answer
0,насыщенный интересный день за любые,назвать избегайте утомительно некоторая именно
100,не торопитесь первая половина дня,избегайте утомительно избегайте тот собеседника
200,день не так уж плох,позволяет легким стремящихся дело проявляете
300,утро будет нервным и напряженным,назвать утомительно умозаключений избегайте сд...
400,благоприятный день для обсуждения важных,избегайте утомительно избегайте такто такто
500,насыщенный и беспокойный день трудно,такто избегайте семейной дело нужны
600,не сердитесь по пустякам поверьте,плодотворный утомительно обществе семейной зап...
701,неплохой день едва ли он,избегайте утомительно избегайте избегайте такто
801,начало дня будет веселым и,легким утомительно стремящихся дело нужны
901,не ждите от окружающих слишком,назвать утомительно вопросах наступает сдвинуть
